As a baseline solution we used this public [notebook](https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train)

In [ ]:
# For right dependencies on kaggle this library was uninstalled
!pip uninstall allennlp -y

Found existing installation: allennlp 2.10.1
Uninstalling allennlp-2.10.1:
  Successfully uninstalled allennlp-2.10.1


### Global settings

In [ ]:
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

Main points concerning the architecture:
* We use pretrained weigts of deberta-v3-base model 
* We fine-tune the last linear layer in respect to 6 desired metrics
* Training via 4-fold cross-validation
* Averaging the score using model from each fold

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
#     wandb=True
    wandb=False
    competition='FB3'
    _wandb_kernel='wallykop'
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="/kaggle/input/debertav3base/"
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=2807
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    train=True

### Imports

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels iterative-stratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Looking in links: ../input/fb3-pip-wheels
Processing /kaggle/input/fb3-pip-wheels/iterative_stratification-0.1.7-py3-none-any.whl


Found existing installation: transformers 4.20.1
Uninstalling transformers-4.20.1:
  Successfully uninstalled transformers-4.20.1


Found existing installation: tokenizers 0.12.1
Uninstalling tokenizers-0.12.1:
  Successfully uninstalled tokenizers-0.12.1


Looking in links: ../input/fb3-pip-wheels
Processing /kaggle/input/fb3-pip-wheels/transformers-4.21.2-py3-none-any.whl
Processing /kaggle/input/fb3-pip-wheels/tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/fb3-pip-wheels


tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


## Definition of utils functions

In [ ]:
from utils import (
    MCRMSE,
    get_score,
    get_logger,
    seed_everything
)

LOGGER = get_logger()  
seed_everything(seed=2807)

# Data load

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


# CV split

In [ ]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    977
1    978
2    978
3    978
dtype: int64

# Deberta tokenizer load

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model, local_files_only=True)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [ ]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/3911 [00:00<?, ?it/s]

max_len: 1429


In [ ]:
# ====================================================
# Dataset
# ====================================================
from dataset import (
    prepare_input,
    TrainDataset,
    collate
)

# Model import

In [ ]:
# ====================================================
# Model
# ====================================================
from model import (
    MeanPooling,
    CustomModel
)

In [ ]:
# ====================================================
# Loss
# ====================================================
from model import RMSELoss

# ====================================================
# Helper functions
# ====================================================
from model import AverageMeter
from utils import asMinutes, timeSince
from main_functions import train_fn, valid_fn

# train loop

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")
    
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

  def get_result(oof_df):
      labels = oof_df[CFG.target_cols].values
      preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
      score, scores = get_score(labels, preds)
      LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

In [ ]:
if __name__ == '__main__':
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        

========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "/kaggle/input/debertav3base/",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

Some weights of the model checkpoint at /kaggle/input/debertav3base/ we

Epoch: [1][0/366] Elapsed 0m 2s (remain 17m 14s) Loss: 2.4604(2.4604) Grad: inf  LR: 0.00002000  
Epoch: [1][20/366] Elapsed 0m 44s (remain 12m 3s) Loss: 0.2929(1.4000) Grad: 132482.9688  LR: 0.00001999  
Epoch: [1][40/366] Elapsed 1m 15s (remain 9m 58s) Loss: 0.1745(0.8305) Grad: 94354.8281  LR: 0.00001998  
Epoch: [1][60/366] Elapsed 1m 54s (remain 9m 33s) Loss: 0.1684(0.6046) Grad: 65162.9609  LR: 0.00001995  
Epoch: [1][80/366] Elapsed 2m 35s (remain 9m 7s) Loss: 0.1224(0.4876) Grad: 82596.6719  LR: 0.00001990  
Epoch: [1][100/366] Elapsed 3m 12s (remain 8m 26s) Loss: 0.1240(0.4257) Grad: 76056.7812  LR: 0.00001985  
Epoch: [1][120/366] Elapsed 3m 51s (remain 7m 48s) Loss: 0.1067(0.3777) Grad: 96725.1562  LR: 0.00001979  
Epoch: [1][140/366] Elapsed 4m 30s (remain 7m 12s) Loss: 0.1338(0.3427) Grad: 66807.3359  LR: 0.00001971  
Epoch: [1][160/366] Elapsed 5m 6s (remain 6m 30s) Loss: 0.2027(0.3150) Grad: 127323.6328  LR: 0.00001962  
Epoch: [1][180/366] Elapsed 5m 39s (remain 5m 47s)

Epoch 1 - avg_train_loss: 0.2093  avg_val_loss: 0.1172  time: 739s
Epoch 1 - Score: 0.4852  Scores: [0.5252723293837455, 0.4603949603507831, 0.4324231774772289, 0.46372561913896837, 0.49361884898336805, 0.5359030621072157]
Epoch 1 - Save Best Score: 0.4852 Model


EVAL: [60/62] Elapsed 1m 10s (remain 0m 1s) Loss: 0.0959(0.1172) 
EVAL: [61/62] Elapsed 1m 10s (remain 0m 0s) Loss: 0.1394(0.1172) 
Epoch: [2][0/366] Elapsed 0m 1s (remain 9m 16s) Loss: 0.0993(0.0993) Grad: 132664.5781  LR: 0.00001809  
Epoch: [2][20/366] Elapsed 0m 43s (remain 11m 52s) Loss: 0.1007(0.1084) Grad: 105600.1641  LR: 0.00001788  
Epoch: [2][40/366] Elapsed 1m 22s (remain 10m 52s) Loss: 0.0884(0.1115) Grad: 49259.2461  LR: 0.00001766  
Epoch: [2][60/366] Elapsed 1m 52s (remain 9m 20s) Loss: 0.0680(0.1073) Grad: 108320.7578  LR: 0.00001744  
Epoch: [2][80/366] Elapsed 2m 26s (remain 8m 35s) Loss: 0.0809(0.1044) Grad: 57301.0391  LR: 0.00001721  
Epoch: [2][100/366] Elapsed 3m 4s (remain 8m 3s) Loss: 0.0765(0.1022) Grad: 109719.0078  LR: 0.00001696  
Epoch: [2][120/366] Elapsed 3m 40s (remain 7m 26s) Loss: 0.0681(0.1022) Grad: 94761.5312  LR: 0.00001671  
Epoch: [2][140/366] Elapsed 4m 17s (remain 6m 51s) Loss: 0.0658(0.1032) Grad: 120961.5547  LR: 0.00001646  
Epoch: [2][160

Epoch 2 - avg_train_loss: 0.1031  avg_val_loss: 0.1122  time: 741s
Epoch 2 - Score: 0.4748  Scores: [0.518505217112889, 0.44943228475971403, 0.4412834160887432, 0.4675421449245012, 0.48459885522931184, 0.48719429333924563]
Epoch 2 - Save Best Score: 0.4748 Model


...

Epoch 3 - avg_train_loss: 0.0941  avg_val_loss: 0.1035  time: 729s
Epoch 3 - Score: 0.4557  Scores: [0.49409977667841654, 0.4513598118150845, 0.4201327976858789, 0.45634952975229137, 0.46602727507509195, 0.4460431630969256]
Epoch 3 - Save Best Score: 0.4557 Model


Epoch: [4][0/366] Elapsed 0m 1s (remain 11m 59s) Loss: 0.0747(0.0747) Grad: 172744.1875  LR: 0.00000692  
Epoch: [4][20/366] Elapsed 0m 42s (remain 11m 32s) Loss: 0.0965(0.0808) Grad: 216642.2344  LR: 0.00000660  
Epoch: [4][40/366] Elapsed 1m 17s (remain 10m 10s) Loss: 0.0774(0.0802) Grad: 168718.2188  LR: 0.00000628  
Epoch: [4][60/366] Elapsed 1m 50s (remain 9m 12s) Loss: 0.0753(0.0826) Grad: 191281.5000  LR: 0.00000596  
Epoch: [4][80/366] Elapsed 2m 23s (remain 8m 23s) Loss: 0.0643(0.0855) Grad: 134483.3438  LR: 0.00000565  
Epoch: [4][100/366] Elapsed 2m 59s (remain 7m 50s) Loss: 0.0815(0.0876) Grad: 126995.8672  LR: 0.00000535  
Epoch: [4][120/366] Elapsed 3m 36s (remain 7m 17s) Loss: 0.0827(0.0883) Grad: 171864.2031  LR: 0.00000504  
Epoch: [4][140/366] Elapsed 4m 10s (remain 6m 39s) Loss: 0.1289(0.0878) Grad: 301860.0000  LR: 0.00000475  
Epoch: [4][160/366] Elapsed 4m 44s (remain 6m 1s) Loss: 0.1077(0.0875) Grad: 207718.3125  LR: 0.00000446  
Epoch: [4][180/366] Elapsed 5m 22

Epoch 4 - avg_train_loss: 0.0855  avg_val_loss: 0.1033  time: 720s
Epoch 4 - Score: 0.4554  Scores: [0.49056861926216083, 0.4522281905279417, 0.4199962631727616, 0.45556276268884976, 0.4675090988182876, 0.44633788321852924]
Epoch 4 - Save Best Score: 0.4554 Model


Epoch: [5][0/366] Elapsed 0m 1s (remain 10m 19s) Loss: 0.0627(0.0627) Grad: 220466.4844  LR: 0.00000192  
Epoch: [5][20/366] Elapsed 0m 46s (remain 12m 50s) Loss: 0.0950(0.0889) Grad: 119457.3281  LR: 0.00000173  
Epoch: [5][40/366] Elapsed 1m 20s (remain 10m 34s) Loss: 0.0815(0.0863) Grad: 161190.3438  LR: 0.00000154  
Epoch: [5][60/366] Elapsed 1m 48s (remain 9m 1s) Loss: 0.0677(0.0840) Grad: 128656.9609  LR: 0.00000136  
Epoch: [5][80/366] Elapsed 2m 24s (remain 8m 29s) Loss: 0.0742(0.0850) Grad: 154329.0469  LR: 0.00000119  
Epoch: [5][100/366] Elapsed 2m 57s (remain 7m 44s) Loss: 0.0556(0.0840) Grad: 130542.9688  LR: 0.00000104  
Epoch: [5][120/366] Elapsed 3m 28s (remain 7m 1s) Loss: 0.0762(0.0836) Grad: 186969.7031  LR: 0.00000089  
Epoch: [5][140/366] Elapsed 4m 7s (remain 6m 34s) Loss: 0.1138(0.0834) Grad: 177147.8125  LR: 0.00000075  
Epoch: [5][160/366] Elapsed 4m 45s (remain 6m 3s) Loss: 0.0664(0.0826) Grad: 116900.3750  LR: 0.00000063  
Epoch: [5][180/366] Elapsed 5m 19s (

Epoch 5 - avg_train_loss: 0.0814  avg_val_loss: 0.1026  time: 731s
Epoch 5 - Score: 0.4538  Scores: [0.4901420855573412, 0.4502627898082984, 0.41639586145381474, 0.45396874903176276, 0.46711645356694903, 0.44471446197691805]
Epoch 5 - Save Best Score: 0.4538 Model
========== fold: 3 result ==========
Score: 0.4538  Scores: [0.4901420855573412, 0.4502627898082984, 0.41639586145381474, 0.45396874903176276, 0.46711645356694903, 0.44471446197691805]
========== CV ==========
Score: 0.4569  Scores: [0.4908865030819827, 0.45083427691211003, 0.41748033615805047, 0.456507211159442, 0.4774611486443539, 0.44828430864670243]


========== fold: 0 result ==========

Score: 0.4535 

========== fold: 1 result ==========

Score: 0.4643  

========== fold: 2 result ==========

Score: 0.4559  

========== fold: 3 result ==========

Score: 0.4538  


**After averaging the scores:**

**========== CV ==========**

**Score: 0.4569**

Each particular error score: 

[0.4908865030819827, 0.45083427691211003, 0.41748033615805047, 0.456507211159442, 0.4774611486443539, 0.44828430864670243] 

for cohesion, syntax, vocabulary, phraseology, grammar, conventions

# Model inference

In [ ]:
# CFG class for inference

class CFG:
    num_workers=4
    path="./"
    config_path=path+'config.pth'
    model="/kaggle/input/debertav3base/"
    gradient_checkpointing=False
    batch_size=24
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]

CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

Loading test sample

In [ ]:
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
test['tokenize_length'] = [len(CFG.tokenizer(text)['input_ids']) for text in test['full_text'].values]
test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)
display(test.head())

,text_id,full_text,tokenize_length
0,000BAD50D026,Do you think students would benefit from being...,441
1,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",488
2,0000C359D63E,when a person has no experience on a job their...,897


In [ ]:
# ====================================================
# Dataset
# ====================================================
from dataset import prepare_input, TestDataSet

# ====================================================
# inference
# ====================================================
from main_functions import inference_fn

from transformers import DataCollatorWithPadding

For final prediction, we average inference from 4 models of cross-validation

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding='longest'),
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions = np.mean(predictions, axis=0)

In [ ]:
test[CFG.target_cols] = predictions
submission = submission.drop(columns=CFG.target_cols).merge(test[['text_id'] + CFG.target_cols], on='text_id', how='left')
display(submission.head())
submission[['text_id'] + CFG.target_cols].to_csv('submission.csv', index=False)

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.936991,2.822174,3.145676,2.973789,2.714838,2.690477
1,000BAD50D026,2.574048,2.419027,2.668774,2.365274,2.152936,2.551006
2,00367BB2546B,3.459992,3.365598,3.575336,3.514001,3.433239,3.274550


This baseline solution give 0.44 score on a public leaderboard